In [1]:


# imports
import imageio
from PIL import Image, ExifTags
from PIL.ExifTags import TAGS
import os
import glob
import subprocess
import pandas as pd

import cProfile
import pstats


# setup
drive = "C:\\"
output_path = os.path.join(drive,'Users','Sydney','Documents','Python','Photography') 
photo_path = os.path.join(drive,'Users','Sydney','Documents','Camera Imports','RAW')
photo_path_test =  os.path.join(drive,'Users','Sydney','Documents','Python','Photography','RAW')
exifToolPath = os.path.join(output_path,'ExifTool','exiftool.exe') #for Windows user have to specify the Exif tool exe path for metadata extraction. 
img_format = ".NEF"

# get all NEF in photo_path
os.chdir(photo_path_test)
files = glob.glob(("*"+img_format))

#get 10 files to test 
subfiles = files[:10]

infoDict = {} #Creating the dict to get the metadata tags
x = 0

for file in subfiles:
    image = subfiles[x]
    x=x+1
    pic = imageio.imread(image)
    type = Image.open(image)
    imgPath = os.path.join(photo_path,image)
    
    for img in imgPath:
        ''' use Exif tool to get the metadata '''
        process = subprocess.Popen([exifToolPath,imgPath],stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True) 

        temp = {}    
        ''' get the tags in dict '''
        for tag in process.stdout:
            line = tag.strip().split(':')
            temp[line[0].strip()] = line[-1].strip()

        infoDict[file]={}
        infoDict[file]['EXIF'] = temp


df = pd.DataFrame.from_dict(infoDict, orient='index')
df2 = pd.json_normalize(df['EXIF'])

lens_analysis = df2[['File Name','Lens ID','Lens Spec','Aperture','Shutter Speed','Auto Focus','Focal Length In 35mm Format','Scale Factor To 35 mm Equivalent','ISO','F Number']]
lens_analysis['Focal Length In 35mm Format'] = (lens_analysis['Focal Length In 35mm Format'].str.extract('(\d+)').astype(float))
lens_analysis['Scale Factor To 35 mm Equivalent'] = lens_analysis['Scale Factor To 35 mm Equivalent'].astype(float)
lens_analysis['Focal Length (Lens)'] = round(lens_analysis['Focal Length In 35mm Format']/lens_analysis['Scale Factor To 35 mm Equivalent'],0)

lens_analysis







C:\Users\Sydney\AppData\Local\Temp\ipykernel_13868\1275506429.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic = imageio.imread(image)


KeyboardInterrupt: 

In [3]:
#%%timeit

# imports
import imageio
from PIL import Image
from PIL.ExifTags import TAGS
import os
import glob
import subprocess
import pandas as pd

# setup
def setup():
    drive = "C:\\"
    output_path = os.path.join(drive,'Users','Sydney','Documents','Python','Photography') 
    photo_path =  os.path.join(drive,'Users','Sydney','Documents','Python','Photography','RAW')
    exifToolPath = os.path.join(output_path,'ExifTool','exiftool.exe') #for Windows user have to specify the Exif tool exe path for metadata extraction. 
    img_format = ".NEF"

    # get all NEF in photo_path
    os.chdir(photo_path)
    files = glob.glob(("*"+img_format))

    #get 10 files to test 
    subfiles = files[:10]
    return  output_path,photo_path,exifToolPath,subfiles

def get_exif_tags(file):
    process = subprocess.Popen([exifToolPath, file],stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True) 

    temp = {}    
    for tag in process.stdout:
        line = tag.strip().split(':')
        temp[line[0].strip()] = line[-1].strip()

    return temp

def get_image_metadata(photo_path, files):
    info_dict = {}
    x = 0
    for file in files:
        image = files[x]
        x = x + 1
        pic = imageio.imread(image)
        type = Image.open(image)
        img_path = os.path.join(photo_path, image)

        info_dict[file] = {}
        info_dict[file]['EXIF'] = get_exif_tags(img_path)

    return info_dict

def convert_to_dataframe(infoDict):
    df = pd.DataFrame.from_dict(infoDict, orient='index')
    df = pd.json_normalize(df['EXIF'])
    df = df[['File Name','Lens ID','Lens Spec','Aperture','Shutter Speed','Auto Focus','Focal Length In 35mm Format','Scale Factor To 35 mm Equivalent','ISO','F Number']]
    df['Focal Length In 35mm Format'] = pd.to_numeric(df['Focal Length In 35mm Format'].replace('[^0-9.]', '', regex=True))
    df['Scale Factor To 35 mm Equivalent'] = df['Scale Factor To 35 mm Equivalent'].astype(float)
    df['Focal Length (Lens)'] = (df['Focal Length In 35mm Format'] / df['Scale Factor To 35 mm Equivalent']).round(0)

    return df



output_path,photo_path,exifToolPath,subfiles = setup()
info_dict = get_image_metadata(photo_path, subfiles)
lens_analysis = convert_to_dataframe(info_dict)

lens_analysis


C:\Users\Sydney\AppData\Local\Temp\ipykernel_13868\3609540450.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic = imageio.imread(image)


,File Name,Lens ID,Lens Spec,Aperture,Shutter Speed,Auto Focus,Focal Length In 35mm Format,Scale Factor To 35 mm Equivalent,ISO,F Number,Focal Length (Lens)
0,DSC_0029.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,5.6,1/100,On,82,1.5,6400,5.6,55.0
1,DSC_0249.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,3.5,1/125,On,27,1.5,1000,3.5,18.0
2,DSC_0448.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,3.5,1/250,On,27,1.5,400,3.5,18.0
3,DSC_0464.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,5.6,1/125,On,82,1.5,9000,5.6,55.0
4,DSC_0555.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,5.6,1/250,On,63,1.5,360,5.6,42.0
5,DSC_0573.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,4.0,1/200,On,36,1.5,800,4.0,24.0
6,DSC_0584.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,4.0,1/250,On,37,1.5,500,4.0,25.0
7,DSC_0632.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,4.5,1/250,On,51,1.5,800,4.5,34.0
8,DSC_0641.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,3.8,1/200,On,36,1.5,800,3.8,24.0
9,DSC_0648.NEF,AF-P DX Nikkor 18-55mm f/3.5-5.6G,18-55mm f/3.5-5.6 G AF-P,4.2,1/125,On,45,1.5,900,4.2,30.0


In [15]:
import os
import subprocess
import imageio

# setup
drive = "C:\\"
output_path = os.path.join(drive,'Users','Sydney','Documents','Python','Photography') 
photo_path = os.path.join(drive,'Users','Sydney','Documents','Camera Imports','RAW')
photo_path_test =  os.path.join(drive,'Users','Sydney','Documents','Python','Photography','RAW')
exifToolPath = os.path.join(output_path,'ExifTool','exiftool.exe') #for Windows user have to specify the Exif tool exe path for metadata extraction. 
img_format = ".NEF"

# get all NEF in photo_path
os.chdir(photo_path_test)
files = glob.glob(("*"+img_format))

#get 10 files to test 
subfiles = files[:10]

infoDict = {} #Creating the dict to get the metadata tags
x = 0

# Declare exifToolPath as a global variable
exifToolPath = '<path_to_exiftool>'

# Iterate over the list of files using enumerate
for i, file in enumerate(subfiles):
    # Use imageio to read the file directly without using Pillow
    pic = imageio.imread(file)
    # Get the file's path
    imgPath = os.path.join(photo_path, file)
    
    # Create an empty dictionary to store the metadata
    temp = {}
    
    # Use exifToolPath and imgPath to get the metadata using subprocess
    process = subprocess.Popen([exifToolPath, imgPath], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True) 
    
    # Parse the metadata and store it in the dictionary
    for tag in process.stdout:
        line = tag.strip().split(':')
        temp[line[0].strip()] = line[-1].strip()
        
    # Store the metadata in the infoDict dictionary
    infoDict[file] = {}
    infoDict[file]['EXIF'] = temp


C:\Users\Sydney\AppData\Local\Temp\ipykernel_204\397836071.py:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic = imageio.imread(file)


In [58]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import os
import glob
import imageio.v2 as imageio
import subprocess
from PIL import Image
import pandas as pd



def get_metadata(exif_tool_path, img_path):
    # use Exif tool to get the metadata 
    process = subprocess.Popen([exif_tool_path, img_path], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True) 

    temp = {}    
    # get the tags in dict 
    for tag in process.stdout:
        line = tag.strip().split(':')
        temp[line[0].strip()] = line[-1].strip()

    return temp
'''
def process_images(photo_path, exif_tool_path, img_format, num_images):
    # get a list of all .NEF files in dir
    os.chdir(photo_path)
    files = glob.glob(("*" + img_format))
    # get subset of num_images files
    subfiles = files[:num_images]

    # run each file through ExifTool and output data into a dictionary
    info_dict = {} # dict to get the metadata tags

    for file in subfiles:
        image = file
        pic = imageio.imread(image)
        type = Image.open(image)
        img_path = os.path.join(photo_path, image)
    
        metadata = get_metadata(exif_tool_path, img_path)
        info_dict[file] = {'EXIF': metadata}
    
    return info_dict
'''

import exiftool

def process_images(photo_path, img_format, num_images, cache=None):
    # get a list of all .NEF files in dir
    os.chdir(photo_path)
    files = glob.glob(("*" + img_format))
    # get subset of num_images files
    subfiles = files[:num_images]

    # run each file through ExifTool and output data into a dictionary
    info_dict = {} # dict to get the metadata tags

    with exiftool.ExifTool() as et:
        for file in subfiles:
            if cache is not None and file in cache:
                # use metadata from cache if available
                metadata = cache[file]
            else:
                # read image data into memory
                img_path = os.path.join(photo_path, file)
                image_data = imageio.imread(img_path)

                # get metadata using exiftool
                metadata = et.get_metadata(img_path)

                # store metadata in cache
                if cache is not None:
                    cache[file] = metadata

            # store metadata in dictionary
            info_dict[file] = {'EXIF': metadata}

    return info_dict



def process_info_dict(info_dict):
    df = pd.DataFrame.from_dict(info_dict, orient='index')
    df = pd.json_normalize(df['EXIF'])
    output = df[['File Name','Lens ID','Lens Spec','Aperture','Shutter Speed','Auto Focus','Focal Length In 35mm Format','Scale Factor To 35 mm Equivalent','ISO','F Number']]    
    
    output['Focal Length In 35mm Format'] = df['Focal Length In 35mm Format'].replace('[^0-9.]', '', regex=True)
    output[['Focal Length In 35mm Format','Scale Factor To 35 mm Equivalent']] = output[['Focal Length In 35mm Format','Scale Factor To 35 mm Equivalent']].apply(pd.to_numeric, errors='coerce')
    output['Focal Length (Lens)'] = (output['Focal Length In 35mm Format'] / output['Scale Factor To 35 mm Equivalent']).apply(lambda x: round(x, 0))
    return output


def main():
    drive = "C:\\"
    path = os.path.join(drive,'Users','Sydney','Documents','Python','Photography') 
    photo_path = os.path.join(path,'RAW')
    exif_tool_path = os.path.join(path,'ExifTool','exiftool.exe') 
    img_format = ".NEF"
    num_images = 10

    info_dict = process_images(photo_path, exif_tool_path, img_format, num_images)
    output_df = process_info_dict(info_dict)
    # display(output_df)
    
    %timeit process_images(photo_path, exif_tool_path, img_format, num_images)
    %timeit process_info_dict(info_dict)
    

if __name__ == "__main__":
    main()




ModuleNotFoundError: No module named 'exiftool'